# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Loading csv of city data
cities_df = pd.read_csv('output_data/cities.csv')
cities_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,nikolskoye,10,RU,1669019385,84,59.7035,30.7861,21.09,4.70
1,1,dikson,31,RU,1669019334,96,73.5069,80.5464,-22.18,4.97
2,2,pekan,100,MY,1669019417,66,3.4836,103.3996,89.71,3.76
3,3,brenes,75,ES,1669019418,97,37.5494,-5.8714,55.85,3.44
4,4,polson,57,US,1669019418,65,47.6936,-114.1632,15.73,1.52


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
#Creating dataframes of latitude and longitude of cities, as well as a humidity series to use as parameters for gmaps
locations = cities_df[['Lat','Lng']]
humidity = cities_df['Humidity']

pandas.core.frame.DataFrame

In [4]:
#setting up gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
#Creating google maps using parameters above, adding a heatmap layer of the cities showing the intensity of humidity in each of the cities of the 
figure_layout = {
'width': '500px',
'height': '500px',
'border': '1px solid black',
'padding': '1px'}
fig = gmaps.figure(layout=figure_layout)
heat_layer = gmaps.heatmap_layer(locations,weights=humidity,dissipating=False,max_intensity=100,point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', padding='1px', width='500px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:

ideal_cities_df = cities_df.loc[(cities_df["Max Temp"]>85) & (cities_df['Wind Speed']<5)]
ideal_cities_df = ideal_cities_df.dropna()
ideal_cities_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
2,2,pekan,100,MY,1669019417,66,3.4836,103.3996,89.71,3.76
121,121,boueni,0,YT,1669019454,75,-12.9025,45.0761,85.98,1.92
265,265,uthal,0,PK,1669019505,38,25.8072,66.6219,86.52,4.70
272,272,pyinmana,42,MM,1669019507,53,19.7333,96.2167,86.49,4.32
281,281,cuamba,6,MZ,1669019510,26,-14.8031,36.5372,97.56,3.36


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
hotel_df = ideal_cities_df
len(hotel_df)

11

In [14]:
#parameters dictionary to use for API calls
params = {"types":"lodging",
         "radius": 10000,
         "keyword":"hotel",
         "key":g_key
         }
#Loop that will identify the nearest hotels to the lat/lng pair we provide from the dataframe of ideal vacation cities
for index,row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    params['location']= f'{lat},{lng}'
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    hotel = requests.get(base_url,params=params)
    hotel = hotel.json()
    response = hotel['results']

    try:
        print(f'The nearest hotel is {response[0]["name"]}.')
        hotel_df.loc[index, "Hotel Name"] = response[0]['name']
    except:
        print("Parameter Missing, Search Result Missing. Skipping....")
    

The nearest hotel is Ancasa Royale Pekan Pahang.
The nearest hotel is ibis Styles Mayotte Aeroport.
The nearest hotel is Ghafoor hotel.
The nearest hotel is Zabu Thiri Hotel.
The nearest hotel is Daania Residence.
The nearest hotel is Wisma Ferdi.
The nearest hotel is Hôtel Résidence KB & Fils.
The nearest hotel is One to Z Hotel.
The nearest hotel is Waves Hotel.
The nearest hotel is JBrownson Hotels.
The nearest hotel is Kernow Charters Towers.


In [15]:
#showing the appended dataframe with the hotel name row
hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
2,2,pekan,100,MY,1669019417,66,3.4836,103.3996,89.71,3.76,Ancasa Royale Pekan Pahang
121,121,boueni,0,YT,1669019454,75,-12.9025,45.0761,85.98,1.92,ibis Styles Mayotte Aeroport
265,265,uthal,0,PK,1669019505,38,25.8072,66.6219,86.52,4.70,Ghafoor hotel
272,272,pyinmana,42,MM,1669019507,53,19.7333,96.2167,86.49,4.32,Zabu Thiri Hotel
281,281,cuamba,6,MZ,1669019510,26,-14.8031,36.5372,97.56,3.36,Daania Residence
305,305,japura,100,ID,1669019519,53,-0.3167,102.3500,90.00,2.48,Wisma Ferdi
323,323,impfondo,60,CG,1669019525,54,1.6381,18.0667,88.47,1.99,Hôtel Résidence KB & Fils
402,402,lafiagi,16,NG,1669019555,21,8.8667,5.4167,87.48,3.96,One to Z Hotel
533,533,umm lajj,3,SA,1669019603,36,25.0213,37.2685,86.23,3.27,Waves Hotel
550,550,idah,92,NG,1669019609,60,7.1065,6.7342,86.47,4.68,JBrownson Hotels


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)

# Display figure
layer_hotels = gmaps.symbol_layer(locations,fill_color='blue',info_box_content=hotel_info,scale=3,stroke_color='black')
fig.add_layer(markers)
fig.add_layer(layer_hotels)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', padding='1px', width='500px'))